In [ ]:
import pandas as pd
import numpy as np
# Builiding a class for computation of reserves using the case_outstanding techique


#attributes of the class are
#df:dataframe having the data
#accident_year:the year in which the accident happend
#reported_year:the year in which the accident was reported
#reported-loss:cummulative reported loss
#paid_reported_loss:cummulative_paid_reported_loss
#case_outstanding:difference between reported loss and paid reported loss;if not there in dataset can be made in the dataframe.

class CaseOutstanding_Reserving:
  def __init__(self,df,accident_year,reported_year,reported_loss,paid_reported_loss,case_outstanding):
    self.df=df
    self.accident_year=accident_year
    self.reported_year=reported_year
    self.reported_loss=reported_loss
    self.paid_reported_loss=paid_reported_loss
    self.case_outstanding=case_outstanding


# This method will help you to build triangle given the parameter
  def d_triangle(self,parameter):
      columns=[]
      max=0
      for i in self.df[self.accident_year].unique():
        a=self.df.groupby(self.accident_year)
        a=a.get_group(i)
        l=len(a)
        if max<l:
          max=l
      final=[]
      for i in range(1,max+1):
        columns.append(12*i)
      j=0
      for i in self.df[self.accident_year].unique():
        check=1
        lis=[]
        while check and j<len(self.df):
          if j+1 <= len(self.df):
            lis.append(self.df.loc[j,parameter])
          if j+1 >= len(self.df):
            check=0
          else:
            curr=self.df[self.accident_year][j]
            next=self.df[self.accident_year][j+1]
          if next != curr:
            check=0
          j=j+1
        if (len(lis)<max):
          for i in range(0,max-len(lis)):
            lis.append(np.NaN)
        final.append(lis)
      bf=pd.DataFrame(final,columns=columns)
      bf["Accident Year"]=self.df[self.accident_year].unique()
      first_column = bf.pop('Accident Year')
      bf.insert(0,"Accident Year",first_column)
      return bf


#This method will help in building incremental triangles especially if you wanna convert cummulative paid claims in to incremental ones
  def incremental_triangles(self):
    final=[]
    for i in range(0,len(self.d_triangle(self.paid_reported_loss))):
      val=[]
      for j in range(0,len(self.d_triangle(self.paid_reported_loss).columns)):
        if j==0 or j==1:
          g=self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j]]
        else:
          if self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j]]!=np.NaN:
            g=self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j]]-self.d_triangle(self.paid_reported_loss).loc[i,self.d_triangle(self.paid_reported_loss).columns[j-1]]
          else:
            g=np.NaN
        val.append(g)
      final.append(val)
    inc=pd.DataFrame(final,columns=self.d_triangle(self.paid_reported_loss).columns)
    return inc


#This will help you to paid_incremental on case ratio
  def paid_tr_on_case_ratio(self):
    final=[]
    li=list(self.d_triangle(self.paid_reported_loss).columns[1:])
    bi=list(self.d_triangle(self.paid_reported_loss).columns[2:])
    for i in range(0,len(self.d_triangle(self.paid_reported_loss))-1):
      val=[]
      for j in range(0,len(li)-1):
        if self.d_triangle(self.paid_reported_loss).loc[i].at[li[j+1]]!=np.NaN and self.d_triangle(self.paid_reported_loss).loc[i].at[li[j]]!=np.NaN:
          g=self.d_triangle(self.paid_reported_loss).loc[i].at[li[j+1]]-self.d_triangle(self.paid_reported_loss).loc[i].at[li[j]]
        else:
          g=np.NaN
        if self.d_triangle(self.case_outstanding).loc[i].at[li[j]]!=np.NaN:
          f=self.d_triangle(self.case_outstanding).loc[i].at[li[j]]
        else:
          f=np.NaN
        if f!=np.NaN and g!=np.NaN:
          val.append(g/f)
        else:
          val.append(np.NaN)
      final.append(val)
    nf=pd.DataFrame(final,columns=bi)
    return nf

# This method will help you to build the remining in case ratios
  def rem_in_case_ratio(self):
    li=list(self.d_triangle(self.case_outstanding).columns[1:])
    bi=li[1:]
    final=[]
    for i in range(0,len(self.d_triangle(self.case_outstanding))-1):
      val=[]
      for j in range(0,len(li)-1):
        if self.d_triangle(self.case_outstanding).loc[i].at[li[j]]!=np.NaN and self.d_triangle(self.case_outstanding).loc[i].at[li[j+1]]!=np.NaN:
          c=self.d_triangle(self.case_outstanding).loc[i].at[li[j+1]]/self.d_triangle(self.case_outstanding).loc[i].at[li[j]]
          val.append(c)
        else:
          val.append(np.NaN)
      final.append(val)
    nf=pd.DataFrame(final,columns=bi)
    return nf


# This method can help you to calculate basic dev factors of case ratios or paid_incremental ratios
  def calc_measures(self,ratio_triangle):#ratio triangle implies case ratio or paid incremental ratios
    columns=list(ratio_triangle.columns)
    final=[]
    for i in columns:
      val=[]
      mean=ratio_triangle[i].mean()
      median=ratio_triangle[i].median()
      val.append(mean)
      val.append(median)
      final.append(val)
    bf=pd.DataFrame(final)
    bf.columns=["Mean","Median"]
    bf["Months"]=(bf.index+2)*12
    bf.transpose()
    return bf

#This method will help you to build the case reserve triangle completely using the measures frame and method like mean and median
  def fill_triangles_case_outstanding(self,measure,method):   #measures=dev-factors of caseReserve ratio, method:mean or median
    dup=self.d_triangle(self.case_outstanding)
    columns=list(dup.columns)
    for i in range(0,len(dup)):
      for j in range(1,len(columns)):
        if dup.isnull().iloc[i,j]:
          dup.loc[i,columns[j]]=round(measure[method][j-2]*dup.loc[i].at[columns[j-1]],3)
    return dup

 #This method will help you to build the paid incremental triangle completely using the measures frame and method like mean and median and filled case reserves.
  def fill_triangles_paid_triangle(self,case_out,paid_inc,measure,method):   #measure:=dev_factors of incremental triangle and method:mean or median
    columns=list(case_out.columns)
    for i in range(0,len(case_out)):
      for j in range(1,len(columns)):
        if paid_inc.isnull().loc[i,columns[j]]:
          paid_inc.loc[i,columns[j]]=(measure[method][j-2]*case_out.loc[i].at[columns[j-1]])
    return paid_inc









